In [ ]:
#Use the following in a bash script
# apt install tesseract-ocr libtesseract-dev poppler-utils

In [ ]:
pip install -r ../requirements.in

In [1]:
from unstructured.partition.pdf import partition_pdf
from unstructured.documents.elements import Title, NarrativeText, Text
from unstructured.chunking.basic import chunk_elements
from typing import List
import os
import weaviate
from weaviate.util import generate_uuid5
import ollama
from pprint import pprint
import json

In [ ]:
ollama.pull("mxbai-embed-large:v1")
ollama.list()

In [2]:
# Use this for now
FILE_PATH = "../docs/Player_s Handbook.pdf"

elements = partition_pdf(filename=FILE_PATH)

Data-loss while decompressing corrupted data


In [ ]:
#save this for later
FILE_PATH = "../docs/Player_s Handbook.pdf"

elements = partition_pdf(filename=FILE_PATH, strategy="hi_res", infer_table_structure=True)

In [9]:
client = weaviate.connect_to_local()
client.connect()

In [13]:
client.collections.delete(name="PHB")

In [41]:
import weaviate.classes as wvc
from weaviate.collections import Collection
from weaviate.client import WeaviateClient

def create_collection(client: WeaviateClient, collection_name: str)-> Collection:
    with client: 
        client.collections.create(
            name=collection_name,
            vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_transformers(),
            vector_index_config=wvc.config.Configure.VectorIndex.hnsw(
                distance_metric=wvc.config.VectorDistances.COSINE # select prefered distance metric
        ),
    )
    collection = client.collections.get(name=collection_name)
    return collection

collection = create_collection(client, 'PHB')


WeaviateClosedClientError: The `WeaviateClient` is closed. Run `client.connect()` to (re)connect!

In [15]:
narrative_texts = [elem for elem in elements if elem.category == "NarrativeText"]
titles = [elem for elem in elements if elem.category == "Title"]
uncat_texts = [elem for elem in elements if elem.category == "UncategorizedText"]


In [ ]:

for text in narrative_texts:
    pprint(text.to_dict())

In [ ]:
for title in titles:
    pprint(title.to_dict())

In [ ]:
for text in uncat_texts:
    pprint(text.to_dict())

In [ ]:
for elem in elements:
    pprint(elem.to_dict())

In [30]:
element_dicts = [chunk.to_dict() for chunk in elements]
pprint(element_dicts[12])


{'element_id': '171ece1a20b9ebcbc9ea79e6a145c909',
 'metadata': {'coordinates': {'layout_height': 792,
                              'layout_width': 612,
                              'points': ((206.2003, 81.61170000000016),
                                         (206.2003, 181.33170000000052),
                                         (369.2218, 181.33170000000052),
                                         (369.2218, 81.61170000000016)),
                              'system': 'PixelSpace'},
              'file_directory': '../docs',
              'filename': 'Player_s Handbook.pdf',
              'filetype': 'application/pdf',
              'languages': ['eng'],
              'last_modified': '2024-04-14T16:19:19',
              'page_number': 4,
              'parent_id': '437aea62a5bd6ad4762e494c81b88532'},
 'text': 'Chapter 8: Combat........................................... 133 The '
         'Battle Grid................................................ 133 How '
         'Comb

In [34]:
embeddings = []

for element in element_dicts[:5]:
    response = ollama.embeddings(model='mxbai-embed-large:v1', prompt=element['text'])
    print(response)
    embedding = response["embedding"]
    embeddings.append(embedding)
    

{'embedding': [0.029506949707865715, -0.31250959634780884, 0.3795894384384155, 0.13557694852352142, -0.1813735067844391, -0.3322491943836212, -0.4111545979976654, -0.0027133747935295105, 0.7953189611434937, 0.20819365978240967, 0.9092506766319275, 0.5145292282104492, 0.6821106672286987, -0.4407135844230652, -0.8744442462921143, -0.7383166551589966, -0.855147659778595, -0.6123324036598206, -1.5352479219436646, 0.053157225251197815, -0.0616745725274086, 0.18974536657333374, -0.6654453277587891, -0.8510355353355408, -0.27767080068588257, 0.6264575719833374, 0.03248577564954758, -0.03348349779844284, 0.7658697962760925, 0.3946005403995514, -0.17256978154182434, -0.2531144917011261, -0.7327723503112793, -0.5201425552368164, -0.3540109694004059, -0.5758035182952881, -0.0984698012471199, -0.007837079465389252, 0.2327495515346527, -0.4337450861930847, 0.2861737608909607, -0.6077984571456909, 0.20824836194515228, -0.30551180243492126, -0.4837956428527832, -0.37836554646492004, 0.156442642211914

In [35]:
embeddings

[[0.029506949707865715,
  -0.31250959634780884,
  0.3795894384384155,
  0.13557694852352142,
  -0.1813735067844391,
  -0.3322491943836212,
  -0.4111545979976654,
  -0.0027133747935295105,
  0.7953189611434937,
  0.20819365978240967,
  0.9092506766319275,
  0.5145292282104492,
  0.6821106672286987,
  -0.4407135844230652,
  -0.8744442462921143,
  -0.7383166551589966,
  -0.855147659778595,
  -0.6123324036598206,
  -1.5352479219436646,
  0.053157225251197815,
  -0.0616745725274086,
  0.18974536657333374,
  -0.6654453277587891,
  -0.8510355353355408,
  -0.27767080068588257,
  0.6264575719833374,
  0.03248577564954758,
  -0.03348349779844284,
  0.7658697962760925,
  0.3946005403995514,
  -0.17256978154182434,
  -0.2531144917011261,
  -0.7327723503112793,
  -0.5201425552368164,
  -0.3540109694004059,
  -0.5758035182952881,
  -0.0984698012471199,
  -0.007837079465389252,
  0.2327495515346527,
  -0.4337450861930847,
  0.2861737608909607,
  -0.6077984571456909,
  0.20824836194515228,
  -0.305511

In [36]:
chunk_embeddings_with_metadata = [
        {
            "id":  None,
            "type": element['type'],
            "title": element['metadata']['filename'],
            "url": "None",
            "content": element['text'],
            "label": "No Label",
            "tokens": len(element['text'].split()),
            "embedding": embedding,
        }
        for element, embedding in zip(element_dicts[:5], embeddings)
    ]

In [39]:
with client: 
    client.collections.delete("PHB")
    client.collections.create("PHB")
    collection = client.collections.get("PHB")
    
    print(collection)

<weaviate.Collection config={
  "name": "PHB",
  "description": null,
  "generative_config": null,
  "inverted_index_config": {
    "bm25": {
      "b": 0.75,
      "k1": 1.2
    },
    "cleanup_interval_seconds": 60,
    "index_null_state": false,
    "index_property_length": false,
    "index_timestamps": false,
    "stopwords": {
      "preset": "en",
      "additions": null,
      "removals": null
    }
  },
  "multi_tenancy_config": {
    "enabled": false
  },
  "properties": [
    {
      "name": "name",
      "description": "This property was generated by Weaviate's auto-schema feature on Sat Apr 20 19:37:07 2024",
      "data_type": "text",
      "index_filterable": true,
      "index_searchable": true,
      "nested_properties": null,
      "tokenization": "word",
      "vectorizer_config": {
        "skip": false,
        "vectorize_property_name": false
      },
      "vectorizer": "text2vec-transformers"
    },
    {
      "name": "dataType",
      "description": "This prop